## 펑션 콜링(function calling) 작동 방식 이해하기 

In [ ]:
api_key = "your-api-key-here"

In [6]:
from llama_index.llms.openai import OpenAI
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
import numpy as np

llm=OpenAI(model="gpt-4o-mini", api_key=api_key)

In [7]:
def add(a,b):
    """ 주어진 두 숫자를 더하고 결과를 출력합니다 """
    return a+b


def mul(a,b):
    """ 주어진 두 숫자를 곱하고 결과를 출력합니다 """
    return a*b

def div(a,b):
    """ 주어진 두 숫자 중 첫번 째 숫자를 두번 째 숫자로 나누고 결과를 출력합니다 """
    return a/b


In [8]:
at=FunctionTool.from_defaults(fn=add)
mt=FunctionTool.from_defaults(fn=mul)
dt=FunctionTool.from_defaults(fn=div)

In [12]:
agent_worker=ReActAgent(
                      name="my_agent",
                      tools=[at,mt,dt],
                      llm=llm,
                      verbose=True
                      )


In [ ]:
response= await agent_worker.run("(77*2+2)를 78로 나눈 값을 계산해 줘")
print(response)

Keyword arguments are not supported when 'run()' is invoked with the 'start_event' parameter. These keyword arguments will be ignored: {'stepwise': False}


Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_ag

## 펑션 콜링 실습 -외부 API를 활용하는 Function Calling Agent

In [18]:
from llama_index.llms.openai import OpenAI
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
import numpy as np

llm=OpenAI(model="gpt-4o-mini", api_key=api_key)

In [19]:
!pip install yfinance==0.2.55

import yfinance as yf

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 46.7 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15636 sha256=457c6bb3a221e4403080a200a807a0d5d7e38784975590d7a45bab51ba658257
  Stored in directory: /Users/paesir/Library/Caches/pip/wheels/cc/bd/6f/664d62c99327abeef7d86489e6631cbf45b56fbf7ef1d6ef00
  Created wheel for peewee: filename=peewee-3.18.2-cp312-cp312-macosx_10_13_universal2.whl size=438002 sha256=d739ce6bfb872b49f53aed3ae83fb9826b1f2a3b75037d866e2d2adbe085a086
  Stored in directory: /Users/paesir/Library/Caches/pip/wheels/d1/df/a9/0202b051c65b11c992dd6db9f2babdd2c44ec7d35d511be5d3
Successfully built multitasking peewee
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [30]:
#yfinance 버전 확인인
print(yf.__version__)

0.2.55


In [31]:
# 미국 주식 최신 종가 호출 
def get_stock_price_us(code):
        ticker = yf.Ticker(f"{code}")
        todays_data = ticker.history(period='1d')
        return round(todays_data['Close'].iloc[0], 2)

# 한국 주식 최신 종가 호출 
def get_stock_price_Korea(code):
        ticker = yf.Ticker(f"{code}.KS")
        todays_data = ticker.history(period='1d')
        return round(todays_data['Close'].iloc[0], 2)


In [45]:

## 'Encountered error: 'Too Many Requests. Rate limited. Try after a while.' 에러가 뜰 경우 함수를 아래와 같이 정의합니다
!pip install curl_cffi
from curl_cffi import requests


# 미국 주식 최신 종가 호출 
def get_stock_price_us(code):
        session = requests.Session(impersonate="chrome")
        ticker = yf.Ticker(f"{code}", session=session)
        todays_data = ticker.history(period='1d')
        return round(todays_data['Close'].iloc[0], 2)

# 한국 주식 최신 종가 호출 
def get_stock_price_Korea(code):
        session = requests.Session(impersonate="chrome")
        ticker = yf.Ticker(f"{code}.KS", session=session)
        todays_data = ticker.history(period='1d')
        return round(todays_data['Close'].iloc[0], 2)

In [46]:
# 도구 등록하기 
stock_k=FunctionTool.from_defaults(fn=get_stock_price_Korea)
stock_u=FunctionTool.from_defaults(fn=get_stock_price_us)


In [47]:
# 에이전트 만들고 쿼리 실행하기 
agent_worker=ReActAgent(
  name="my_agent",
  tools=[stock_k, stock_u], 
  llm=llm, 
  verbose=True,
  allow_parallel_tool_calls=False
  )

In [48]:
response= await agent_worker.run("TESLA 최신 종가가 어떻게 돼?")
print(response)


Keyword arguments are not supported when 'run()' is invoked with the 'start_event' parameter. These keyword arguments will be ignored: {'stepwise': False}


Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent
TESLA의 최신 종가는 347.79입니다.


In [49]:
response= await agent_worker.run("삼성전자의 최신 종가가 어떻게 돼?")
print(response)


Keyword arguments are not supported when 'run()' is invoked with the 'start_event' parameter. These keyword arguments will be ignored: {'stepwise': False}


Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent
삼성전자의 최신 종가는 73,400원입니다.


### 주식 최신 종가를 날짜 정보와 함께 알고 싶을 때 

In [61]:
def get_stock_price_us(code):
    """미국 주식의 최신 종가와 날짜를 조회합니다."""
    ticker = yf.Ticker(f"{code}")
    todays_data = ticker.history(period='1d')
    
    if not todays_data.empty:
        close_price = round(todays_data['Close'].iloc[0], 2)
        close_date = todays_data.index[0].strftime('%Y-%m-%d')
        return {
            "price": close_price,
            "date": close_date,
            "ticker": code
        }
    else:
        return {"error": "데이터를 찾을 수 없습니다"}

def get_stock_price_Korea(code):
    """한국 주식의 최신 종가와 날짜를 조회합니다."""
    ticker = yf.Ticker(f"{code}.KS")
    todays_data = ticker.history(period='1d')
    
    if not todays_data.empty:
        close_price = round(todays_data['Close'].iloc[0], 2)
        close_date = todays_data.index[0].strftime('%Y-%m-%d')
        return {
            "price": close_price,
            "date": close_date,
            "ticker": code
        }
    else:
        return {"error": "데이터를 찾을 수 없습니다"}

In [62]:
stock_k=FunctionTool.from_defaults(fn=get_stock_price_Korea)
stock_u=FunctionTool.from_defaults(fn=get_stock_price_us)

In [63]:
agent_worker=ReActAgent(
  name="my_agent",
  tools=[stock_k, stock_u], 
  llm=llm, 
  verbose=True,
  allow_parallel_tool_calls=False
  )

In [64]:
response= await agent_worker.run("TESLA 최신 종가가 어떻게 돼?")
print(response)

Keyword arguments are not supported when 'run()' is invoked with the 'start_event' parameter. These keyword arguments will be ignored: {'stepwise': False}


Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent
TESLA의 최신 종가는 347.79달러이며, 날짜는 2025년 9월 10일입니다.


In [65]:
response= await agent_worker.run("삼성전자 최신 종가가 어떻게 돼?")
print(response)

Keyword arguments are not supported when 'run()' is invoked with the 'start_event' parameter. These keyword arguments will be ignored: {'stepwise': False}


Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent
삼성전자의 최신 종가는 73,400원이며, 날짜는 2025년 9월 11일입니다.


## 펑션 콜링 실습 - Context-Augmented Function Calling Agent

In [66]:
#데이터 다운로드 
import os
import urllib.parse
import requests
import re

urls=[
    "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch08/data/%5B%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90%5D%EC%82%AC%EC%97%85%EB%B3%B4%EA%B3%A0%EC%84%9C_2022.pdf",
    "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch08/data/%5B%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90%5D%EC%82%AC%EC%97%85%EB%B3%B4%EA%B3%A0%EC%84%9C_2023.pdf",
    "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch08/data/%5B%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90%5D%EC%82%AC%EC%97%85%EB%B3%B4%EA%B3%A0%EC%84%9C_2024.pdf"
    ]

# 각 파일 다운로드
for url in urls:
    encoded_filename = url.split("/")[-1]  # URL에서 파일명 추출
    decoded_filename = urllib.parse.unquote(encoded_filename) # 한글 파일명 복원
    response = requests.get(url)
    if response.status_code == 200:
# 임시 파일명으로 저장
        temp_filename = "temp_download_file" + os.path.splitext(decoded_filename)[1] 
        with open(temp_filename, 'wb') as f:
            f.write(response.content)            
        os.rename(temp_filename, decoded_filename) # 파일명 변경
        print(f"완료: {decoded_filename} 다운로드 완료")
    else:
        print(f"오류: {url} 다운로드 실패 (상태 코드: {response.status_code})\n")


완료: [삼성전자]사업보고서_2022.pdf 다운로드 완료
완료: [삼성전자]사업보고서_2023.pdf 다운로드 완료
완료: [삼성전자]사업보고서_2024.pdf 다운로드 완료


In [68]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.settings import Settings

Settings.llm = OpenAI(model="gpt-4o", api_key=api_key)
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small", api_key=api_key)

In [69]:

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex  
)
from llama_index.core.tools import QueryEngineTool

In [70]:
# load data
S2024_docs = SimpleDirectoryReader(
    input_files=["./data/[삼성전자]사업보고서_2024.pdf"]
    ).load_data()
S2023_docs = SimpleDirectoryReader(
    input_files=["./data/[삼성전자]사업보고서_2023.pdf"]
    ).load_data()
S2022_docs = SimpleDirectoryReader(
    input_files=["./data/[삼성전자]사업보고서_2022.pdf"]
    ).load_data()

# build index
S2024_index = VectorStoreIndex.from_documents(S2024_docs)
S2023_index = VectorStoreIndex.from_documents(S2023_docs)
S2022_index = VectorStoreIndex.from_documents(S2022_docs)
    

In [72]:
S2024_engine = S2024_index.as_query_engine(similarity_top_k=3)
S2023_engine = S2023_index.as_query_engine(similarity_top_k=3)
S2022_engine = S2022_index.as_query_engine(similarity_top_k=3)

In [73]:
query_engine_tools = [
    QueryEngineTool.from_defaults(
        query_engine=S2024_engine,
        name="samsung_2024",
        description=(
            "삼성전자의 2024년 재무상태에 대해 정보 제공해 주세요."
            "도구에 입력할 때 자세한 일반 텍스트 질문을 사용합니다."
            "실적 분석할 때 보고서를 충분히 검토한 후 답변해 주세요"
        ),
    ),
    QueryEngineTool.from_defaults(
        query_engine=S2023_engine,
        name="samsung_2023",
        description=(
             "삼성전자의 2023년 재무상태에 대해 정보 제공해 주세요."
            "도구에 입력할 때 자세한 일반 텍스트 질문을 사용합니다."
            "실적 분석할 때 보고서를 충분히 검토한 후 답변해 주세요"
        ),
    ),
    QueryEngineTool.from_defaults(
        query_engine=S2022_engine,
        name="samsung_2022",
        description=(
             "삼성전자의 2022년 재무상태에 대해 정보 제공해 주세요."
            "도구에 입력할 때 자세한 일반 텍스트 질문을 사용합니다."
            "실적 분석할 때 보고서를 충분히 검토한 후 답변해 주세요"
        ),
    ),
]

In [77]:
# agent 생성
from llama_index.core.agent import ReActAgent

agent = ReActAgent(
    name="my_agent",
    tools=query_engine_tools,  # tools 대신 query_engine_tools 사용
    llm=OpenAI(model="gpt-4o-mini", api_key=api_key),
    verbose=True,
    )


In [79]:
response = await agent.run(
    "삼성전자의 2022년, 2023년, 2024년 연결기준 매출액을 각각 알려주세요. "
    "각 연도의 매출액을 원화 단위로 제시해 주세요."
)
print(response)

Keyword arguments are not supported when 'run()' is invoked with the 'start_event' parameter. These keyword arguments will be ignored: {'stepwise': False}


Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_ag

In [80]:
response= await agent.run("삼성전자의 2022년, 2023년, 2024년 매출액과 영업이익을 알려줘. 그리고 2023년에는 전년대비 줄어든 매출액이 2024년에는 다시 회복된 이유를 뭐라고 분석하고 있는지 찾아줘")
print(response)

Keyword arguments are not supported when 'run()' is invoked with the 'start_event' parameter. These keyword arguments will be ignored: {'stepwise': False}


Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_ag

In [81]:
response=await agent.run("삼성전자의 2023년 매출액이 2022년에 비해 많이 줄었는데, 2024년에는 다시 회복됐어. 이유를 뭐라고 분석하고 있어? 보고서를 충분히 검토하고 답변해 줘")
print(response)

Keyword arguments are not supported when 'run()' is invoked with the 'start_event' parameter. These keyword arguments will be ignored: {'stepwise': False}


Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_ag